# ALBERT for aptamer-pair classification

## Set dependancies

In [2]:
import torch
import torch.nn as nn
import torch.onnx
import os
import matplotlib.pyplot as plt
import copy
import torch.optim as optim
import random
import numpy as np
import pandas as pd
import ruamel.yaml
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
import transformers
import logging
from transformers import logging

logging.set_verbosity_error() #  to skip errors

os.environ["TOKENIZERS_PARALLELISM"] = "false"
config_name = 'config.yaml'

In [ ]:
with open(config_name, 'r') as stream:
    try:
        yaml = ruamel.yaml.YAML()
        config = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

## Load your dataset

In [4]:
path_to_train_data = config['Datasets']['train']
path_to_val_data = config['Datasets']['val']
path_to_test_data = config['Datasets']['test']


df_train = pd.read_csv(path_to_train_data)
df_val = pd.read_csv(path_to_val_data)
df_test = pd.read_csv(path_to_test_data)

#  Check the data
print(df_train.shape, df_val.shape, df_test.shape)

In [6]:
df_train.head()

,Sequence1,Sequence2,Label
0,GCGGTAACCGGTGCT,GGAGAACAACAACTT,0
1,ATCAGATACTATGCG,TACAGAACTAGCAGG,0
2,AGAGTGCCCTGTCCC,ATCTGCTAACCAGTC,0
3,ATTCCCAAGCTGCGA,GTACCCGGGATGTTA,0
4,GCGAGCTCATGCACA,GGGCGCCATAGTTCG,0


## Classes and functions

In [7]:
class CustomDataset(Dataset):

    def __init__(self, data, maxlen=32, with_labels=True, bert_model='albert-base-v2'):
        self.data = data 
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model, return_dict=False)  
        self.maxlen = maxlen
        self.with_labels = with_labels 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        seq1 = str(self.data.loc[index, 'Sequence1'])
        seq2 = str(self.data.loc[index, 'Sequence2'])

        # Tokenizing input of two sequences "sentence" to get token ids, attention masks and token type ids
        encoded_pair = self.tokenizer(seq1, seq2, 
                                      padding='max_length',  #  Pad input if tokenized input is shorter than max_length
                                      truncation=True,  #  Truncate in case tokenized input surpasses length 32
                                      max_length=self.maxlen,  
                                      return_tensors='pt')  #  Return torch.Tensor objects
        
        #  Tensors 
        token_ids = encoded_pair['input_ids'].squeeze(0)  #  Of token ids
        attn_masks = encoded_pair['attention_mask'].squeeze(0)  #  "0" - padded values and "1" - other values
        token_type_ids = encoded_pair['token_type_ids'].squeeze(0)  #  "0" - 1st sentence tokens, "1" - 2nd sentence tokens

        if self.with_labels:  # True if the dataset has labels, for training or evaluating models goodness metrics, where we have labels of "correct class"
            label = self.data.loc[index, 'Label']
            return token_ids, attn_masks, token_type_ids, label  
        else: #  In case we are running prediction on never seen aptamers
            return token_ids, attn_masks, token_type_ids

In [8]:
class Model(nn.Module):

    def __init__(self, bert_model="albert-base-v2", freeze_bert=False):
        super(Model, self).__init__()
        self.bert_layer = AutoModel.from_pretrained(bert_model, return_dict=False)
        self.config = self.bert_layer.config #  this line helps to save its configs later on
        
        #  More information on number of models hidden size can be found https://huggingface.co/transformers/pretrained_models.html
        hidden_size = config['Model']['hidden_size']

        #  Freeze every bert layer except the classification layer 
        if freeze_bert:
            for l in self.bert_layer.parameters():
                l.requires_grad = False

        #  Classification layer
        self.cls_layer = nn.Linear(hidden_size, config['Model']['number_labels'])
        self.dropout = nn.Dropout(p=config['Model']['dropout_rate'])

    @autocast()  #  run in mixed precision, it helps to save some precious time :)
    def forward(self, input_ids, attn_masks, token_type_ids):

        #  Feeding the inputs of `CustomDataset` to the BERT-based model to obtain contextualized representations
        cont_reps, pooler_output = self.bert_layer(input_ids, attn_masks, token_type_ids)

        #  Feeding to the classifier layer the last layer hidden-state of the [CLS] token further processed by a
        #  Linear Layer and a Tanh activation (which determines the class of aptamers pair)
        logits = self.cls_layer(self.dropout(pooler_output))

        return logits

In [9]:
#  Seed helps to recreate your results
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    

def evaluate_loss(net, device, criterion, dataloader):
    net.eval()
    mean_loss = 0
    count = 0

    with torch.no_grad():
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(dataloader)):
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
            logits = net(seq, attn_masks, token_type_ids)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            count += 1

    return mean_loss / count

In [10]:
def train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate):
    best_loss = np.Inf
    nb_iterations = len(train_loader)
    print_every = nb_iterations // config['Random']['print_every']  #  To print fine-tuning results less often if needed
    scaler = GradScaler()
    
    iters = []
    train_losses = []
    val_losses = []

    for ep in range(epochs):

        net.train()
        running_loss = 0.0
        for it, (seq, attn_masks, token_type_ids, labels) in enumerate(tqdm(train_loader)):

            # Converting to cuda tensors
            seq, attn_masks, token_type_ids, labels = \
                seq.to(device), attn_masks.to(device), token_type_ids.to(device), labels.to(device)
    
            # Enables autocasting for the forward pass (model + loss)
            with autocast():
                # Obtaining the logits from the model
                logits = net(seq, attn_masks, token_type_ids)

                # Computing loss
                loss = criterion(logits.squeeze(-1), labels.float())
                loss = loss / iters_to_accumulate  # Normalize the loss because it is averaged

            # Backpropagating the gradients. Calls backward() on scaled loss to create scaled gradients.
            scaler.scale(loss).backward()

            if (it + 1) % iters_to_accumulate == 0:
                # Optimization step
                scaler.step(opti)
                # Updates the scale for next iteration.
                scaler.update()
                # Adjust the learning rate based on the number of iterations.
                lr_scheduler.step()
                # Clear gradients
                opti.zero_grad()

            running_loss += loss.item()

            if (it + 1) % print_every == 0:  # Print training loss information
                print("Iteration {}/{} of epoch {} complete. Loss : {} "
                      .format(it+1, nb_iterations, ep+1, running_loss / print_every))
                
                train_losses.append(running_loss/print_every)
                running_loss = 0.0


        val_loss = evaluate_loss(net, device, criterion, val_loader)  # Compute validation loss
        val_losses.append(val_loss)
        
        print("Epoch {} complete! Validation Loss : {}".format(ep+1, val_loss))

        if val_loss < best_loss:

            print("Best validation loss improved from {} to {}".format(best_loss, val_loss))
            net_copy = copy.deepcopy(net)  # save a copy of the model
            best_loss = val_loss
        
    # Saving the model
    path_to_model='model/{}_{}_{}.pt'.format(bert_model, lr, round(best_loss, 4)) #  renames path of model to instantly use it to evaluate metrics
    path_to_model_evaluation = './datasets/model_validation/{}_{}_{}.csv'.format(bert_model, lr, round(best_loss, 4)) # Path to model fine-tuning losses
    path_to_model_test = './datasets/model_validation/pred-{}_{}_{}.csv'.format(bert_model, lr, round(best_loss, 4)) #  Path to model predicted test dataset
    
    #  Following val/test losses to graph it later
    model_training = './datasets/model_validation/{}_{}_{}_training.csv'.format(bert_model, lr, round(best_loss, 4))
    losses = pd.DataFrame({'Train loss': train_losses, 'Val loss': val_losses})
    losses.to_csv(model_training)
    
    config['Random']['path_to_model'] = path_to_model
    config['Random']['path_to_model_evaluation'] = path_to_model_evaluation
    config['Random']['path_to_model_test'] = path_to_model_test

    torch.save(net_copy.state_dict(), path_to_model)
    print("The model has been saved in {}".format(path_to_model))

    #  To save PyTorch model to .bin file with config.json run:
    #configs = net.congif
    #net.save_pretrained('Model-Directory')
    
    del loss
    torch.cuda.empty_cache()

## Hyperparameters for training

In [11]:
bert_model = config['Model']['bert_model']  # any of previously defined BERT alternatives :'albert-base-v2', 'albert-large-v2', 'albert-xlarge-v2', 'albert-xxlarge-v2' and others
freeze_bert = config['Model']['freeze_bert']  # if True, freeze the encoder weights and only update the classification layer weights
maxlen = config['Model']['max_len']        # maximum length of the tokenized input sentence pair : if greater than "maxlen", the input is truncated and else if smaller, the input is padded
bs = config['Model']['batch_size']                # batch size
iters_to_accumulate = config['Model']['iters_to_accumulate']  # the gradient accumulation adds gradients over an effective batch of size : bs * iters_to_accumulate. If set to "1", you get the usual batch size
lr = config['Model']['learning_rate']                # learning rate
epochs = config['Model']['epochs']             # number of training epochs

In [ ]:
#  Set all seeds to make reproducible results
set_seed(2021)
 
#  Creating instances of training and validation set
print("Reading training and validation data...")
train_set = CustomDataset(df_train, maxlen, bert_model)
val_set = CustomDataset(df_val, maxlen, bert_model)

#  Creating isntances for model training
train_loader = DataLoader(train_set, batch_size=bs, num_workers=2)
val_loader = DataLoader(val_set, batch_size=bs, num_workers=2)
 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Model(bert_model, freeze_bert=freeze_bert)

#  In case you have more than 1 GPU
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)

net.to(device)
   
criterion = nn.BCEWithLogitsLoss()
opti = AdamW(net.parameters(), lr=lr, weight_decay=1e-2) #  More on AdamW can be found https://towardsdatascience.com/7-tips-to-choose-the-best-optimizer-47bb9c1219e 
num_warmup_steps = config['Model']['num_warmup_steps'] #  The number of steps for the warmup phase.
num_training_steps = epochs * len(train_loader)  #  The total number of training steps
t_total = (len(train_loader) // iters_to_accumulate) * epochs  #  Necessary to take into account Gradient accumulation
lr_scheduler = get_linear_schedule_with_warmup(optimizer=opti, num_warmup_steps=num_warmup_steps, num_training_steps=t_total)
                         
train_bert(net, criterion, opti, lr, lr_scheduler, train_loader, val_loader, epochs, iters_to_accumulate)

#  Dump changed configuration parameters in train_bert
with open('config.yaml', 'w') as conf:
    yaml.dump(config, conf)

# Prediction

In [16]:
#  Simply converting logit tensors to probabilities arrays
def get_probs_from_logits(logits):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    return probs.detach().cpu().numpy()

#  Predict from probabilities and move results to csv
def test_prediction(net, device, aptamerDataFrame, dataloader, with_labels, result_path=config['Random']['path_to_model_test']):
    net.eval()
    probs_all = []
    nb_iterations = len(dataloader)
    
    with torch.no_grad():
        if with_labels:
            for it, (seq, attn_masks, token_type_ids, label) in enumerate(tqdm(dataloader)):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
                
        else:
            for it, (seq, attn_masks, token_type_ids, token_ids) in enumerate(tqdm(dataloader)):
                seq, attn_masks, token_type_ids = seq.to(device), attn_masks.to(device), token_type_ids.to(device)
                logits = net(seq, attn_masks, token_type_ids)
                probs = get_probs_from_logits(logits.squeeze(-1)).squeeze(-1)
                probs_all += probs.tolist()
            
    y_hat = [round(x) for x in probs_all]     
    df2 = pd.DataFrame({'y_hat': y_hat, 'prob': probs_all})
    df = pd.concat([aptamerDataFrame, df2], axis=1)
    df.to_csv(result_path)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Reading test data...")
test_set = CustomDataset(df_test, maxlen, bert_model)
test_loader = DataLoader(test_set, batch_size=bs, num_workers=4)

model = Model(bert_model)
if torch.cuda.device_count() > 1:  # if multiple GPUs
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

print("Loading the weights of the model...")
model.load_state_dict(torch.load(config['Random']['path_to_model']))
model.to(device)

print("Predicting on test data...")
test_prediction(net=model
                , device=device
                , dataloader=test_loader
                , aptamerDataFrame=df_test
                , with_labels = True)

print("Predictions are available in : {}".format(config['Random']['path_to_model_test']))

#  Convert model to ONNX


In [ ]:
if config['Model']['convert_to_onnx']:
    sequences = {'AGCTAAGCTAAGCTA':'AAGACTGACAGCTAA'}
    sequences=pd.DataFrame(sequences.items(), columns=['Sequence1', 'Sequence2'])
    dataset = CustomDataset(
        data = sequences,
        maxlen = config['Model']['max_len'],
        with_labels = False #sulyginti su class cusotmedatasets
        )
    
    model = Model(bert_model)
    
    model.load_state_dict(torch.load(config['Random']['path_to_model']))
    model.to(device)
    model.eval()
    #defining model inputs, which are ids, kmask and toke type ids (it comes from Model Class)
    input_ids= dataset[0][0].unsqueeze(0).cuda()
    attn_masks = dataset[0][0].unsqueeze(0).cuda()
    token_type_ids = dataset[0][0].unsqueeze(0).cuda()
        
    torch.onnx.export(
        model, #.module if paralized
        (input_ids, attn_masks, token_type_ids),
        "albert-base-albumin.onnx",
        input_names=["input_ids", "attn_masks", "token_type_ids"], 
        output_names=["output"],
        #which inputs have dynamical axes
        verbose=True,
        dynamic_axes={
            "input_ids": {0: "batch_size"},
            "attn_masks": {0: "batch_size"},
            "token_type_ids": {0: "batch_size"},
            "output": {0: "batch_size"},
        },
        opset_version=10,
        )    